# Critique extraction

In [3]:
from utils import load_openai_key

load_openai_key()

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_msg = ChatPromptTemplate.from_template("""
    Extract all the issues mentioned in the review and return them as a list.

    Review:
    {input}
""")

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Review(BaseModel):
    """Collection of issues mentioned in a review"""
    critique_points: list[str] = Field(description="List of issues mentioned in the review")

In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(Review)

In [7]:
chain = prompt_msg | llm

In [19]:
from pathlib import Path


reviews = list(Path('ReviewFetcher/raw_reviews').glob('*.txt'))
len(reviews)

30

In [20]:
reviews[0].read_text().replace("'", '"')

'{"title": "3,0 von 5 Sternen\\nToll, aber!", "text": "Meine Kinder lieben den Würfel, leider ist er vermutlich nicht dafür gebaut, wirklich lange und ständig verwendet zu werden. Die geklebte Folie löst sich nun ab.", "rating": "3,0 von 5 Sternen", "date": "25. Mai 2024"}'

In [21]:
import json


review = json.loads(reviews[0].read_text().replace("'", '"'))

In [22]:
review

{'title': '3,0 von 5 Sternen\nToll, aber!',
 'text': 'Meine Kinder lieben den Würfel, leider ist er vermutlich nicht dafür gebaut, wirklich lange und ständig verwendet zu werden. Die geklebte Folie löst sich nun ab.',
 'rating': '3,0 von 5 Sternen',
 'date': '25. Mai 2024'}

In [32]:
from dataclasses import dataclass
from datetime import datetime


@dataclass
class ReviewInput:
    title: str
    text: str
    rating: int
    date: datetime
    
    @property
    def prompt_txt(self) -> str:
        return f"Title: {self.title}\nText: {self.text}\nRating: {self.rating}/5\nDate: {self.date.strftime('%Y-%m-%d')}"


In [39]:
import locale


def parse_to_date_object(date_str: str) -> datetime:
    locale.setlocale(locale.LC_TIME, 'de_DE')
    date_obj = datetime.strptime(date_str, "%d. %B %Y")
    return date_obj

title = " ".join(review['title'].split('\n')[1:])
rating = int(review['rating'][0])
date = parse_to_date_object(review['date'])

review_input = ReviewInput(title, review['text'], rating, date)

In [40]:
print(review_input.prompt_txt)

Title: Toll, aber!
Text: Meine Kinder lieben den Würfel, leider ist er vermutlich nicht dafür gebaut, wirklich lange und ständig verwendet zu werden. Die geklebte Folie löst sich nun ab.
Rating: 3/5
Date: 2024-05-25


In [41]:
chain.invoke({"input": review_input.prompt_txt})

Review(critique_points=['Nicht für dauerhafte Verwendung geeignet', 'Geklebte Folie löst sich ab'])